In [2]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
# importing dataset
train = pd.read_csv("train.csv")
train = train.set_index('ID')

In [4]:
test = pd.read_csv("test.csv")
test = test.set_index('ID')

In [5]:
# train.head()

In [5]:
# test.head()

In [6]:
# check for datatypes
# train.dtypes

In [7]:
# # check for missing values
# train.isna().sum()

In [8]:
# test.isna().sum()

In [9]:
# fig=plt.subplots(figsize=(8,8))
# corrMatrix = train.corr()
# sns.heatmap(corrMatrix, annot=True)
# plt.show()

In [10]:
# import random
# # filling missing values
# train = train.fillna({True:"Yes",False:"No"} [random.randint(0, 1)==0])
# test= test.fillna({True:"Yes",False:"No"} [random.randint(0, 1)==0])

In [6]:
# filling missing values
train = train.fillna(train['Credit_Product'].value_counts().index[0])
test= test.fillna(test['Credit_Product'].value_counts().index[0])

In [12]:
# # check for missing values after modifying
# train.isna().sum()

In [13]:
# test.isna().sum()

In [14]:
# # visualization
# train.hist();

In [7]:
# encoding categorical values for training set
lb_make = LabelEncoder()

for i in train.columns:
    if train[i].dtype == 'object':
        train[i] = lb_make.fit_transform(train[i])

train.head() 

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
ID,,,,,,,,,,
NNVBBKZB,0,73,18,1,2,43,0,1045696,0,0
IDD62UNG,0,30,27,2,0,32,0,581988,0,0
HD3DSEMC,0,56,18,3,2,26,0,1484315,1,0
BF3NC7KV,1,34,20,2,0,19,0,470454,0,0
TEASRWXV,0,30,32,2,0,33,0,886787,0,0


In [8]:
# encoding categorical values for testing set
lb_make = LabelEncoder()

for i in test.columns:
    if test[i].dtype == 'object':
        test[i] = lb_make.fit_transform(test[i])

test.head() 

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active
ID,,,,,,,,,
VBENBARO,1,29,4,1,0,25,1,742366,0
CCMEWNKY,1,43,18,1,1,49,0,925537,0
VK3KGA9M,1,31,20,2,0,14,0,215949,0
TT8RPZVC,1,29,22,1,0,33,0,868070,0
SHQZEYTZ,0,29,20,1,0,19,0,657087,0


In [9]:
# splitting data into training and testing
X = train.drop('Is_Lead', axis=1)
y = train['Is_Lead']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [20]:
# feature scaling
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)
# test = sc.transform(test)

In [20]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn import svm 
# from sklearn.naive_bayes import GaussianNB
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier

# from sklearn.metrics import roc_auc_score
# models = [LogisticRegression(solver="liblinear"), KNeighborsClassifier(),  GaussianNB(), DecisionTreeClassifier(),
#           RandomForestClassifier(), XGBClassifier()]
# for model in models:
#     model.fit(X_train, y_train)
#     predictions = model.predict_proba(X_test)
#     print(type(model).__name__)
#     print("    AUC_SCORE", roc_auc_score(y_test, predictions[:,1]))

In [10]:
params = {
  'colsample_bynode': 0.8,
  'learning_rate': 1,
  'max_depth': 5,
  'num_parallel_tree': 100,
  'objective': 'binary:logistic',
  'subsample': 0.8,
  'tree_method': 'gpu_hist'
}

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
xgb = XGBClassifier(eta=0.3,objective= 'binary:logistic',tree_method= 'gpu_hist',num_parallel_tree= 500,)
xgb.fit(X_train, y_train)
predictions = xgb.predict_proba(X_test)
# print(type(model).__name__)
print("    AUC_SCORE", roc_auc_score(y_test, predictions[:,1]))

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:53:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [39]:
result = xgb.predict_proba(test)

In [40]:
df=pd.read_csv("test.csv")
ID = df["ID"]
res = pd.DataFrame(ID)
res.columns = ["ID"]
abc= pd.DataFrame(result)
res["Is_Lead"] = abc[1]
res.head()
res.to_csv("submission.csv",index=False)
#res["Is_Lead"] = result[0]
# res.to_csv("submission.csv",index=False)